In [2]:
%matplotlib inline

# useful packages
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx
import time
import pandas as pd

from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA, COBYLA, SLSQP
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit.quantum_info import Statevector
from qiskit import Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram
from qiskit.visualization import plot_state_city, plot_bloch_multivector
from qiskit.visualization import plot_state_paulivec, plot_state_hinton
from qiskit.visualization import plot_state_qsphere
import qiskit

In [3]:
# graph generation

def draw_graph(G, colors, pos):
        default_axes = plt.axes(frameon=True)
        nx.draw_networkx(G, node_color=colors, node_size=600, alpha=0.8, ax=default_axes, pos=pos)
        edge_labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)

def graph_generation(dim, periodic  = False):
    # Generating the system
    N = dim
    G1 = nx.grid_2d_graph(N,N)
    pos = dict( (n, n) for n in G1.nodes() )
    labels = dict( ((i, j), i * N + j) for i, j in G1.nodes() )
    # Transform to weighted graph:
    n = N**2
    V = np.arange(0, N, 1)

    E =[]

    tuples = []
    other_tup = []
    point_edge_map = []
    other_map = []
    relations = []

    for edge in G1.edges:
        point1 = edge[0]
        strpoint1 = str(point1[0]) + str(point1[1])
        point2 = edge[1]
        strpoint2 = str(point2[0]) + str(point2[1])
        if not strpoint1 in point_edge_map:
            point_edge_map.append(strpoint1)
            tuples.append(edge[0])
        if not strpoint2 in other_map:
            other_map.append(strpoint2)
            other_tup.append(edge[1])
        relations.append((strpoint1, strpoint2))

     # add periodic boundary conditions
    if periodic:
        grid = np.zeros((dim, dim)).astype('int64').tolist()
        for i in range(dim):
            for j in range(dim):
                pt = str(i) + str(j)
                grid[i][j] = pt
        for i in range(len(grid)):
            relations.append((grid[0][i], grid[dim-1][i]))
            relations.append((grid[i][0], grid[i][dim-1]))  
        
    tuples.append(other_tup[len(other_tup) - 1])
    point_edge_map.append(other_map[len(other_map) - 1])

    dic = {}
    for i in range(len(point_edge_map)):
        dic[i] = point_edge_map[i]

    point_edge_map = np.array(point_edge_map)
    relations = np.array(relations)

    for relation in relations:
        ver1 = relation[0]
        ver2 = relation[1]
        point1 = np.where(point_edge_map == ver1)[0][0]
        point2 = np.where(point_edge_map == ver2)[0][0]
        edge = (point1, point2, 1.0)
        E.append(edge)

    G = nx.Graph()

    G.add_nodes_from(V)

    G.add_weighted_edges_from(E)

    colors = ["b" for node in G.nodes()]
    pos = dict()
    for i in range(n):
        pos[i] = tuples[i]


    # draw_graph(G, colors, pos)
    
    # Matrix Representation of Graph
    w = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            temp = G.get_edge_data(i, j, default=0)
            if temp != 0:
                w[i, j] = temp["weight"]
                
    return G, w, pos

In [4]:
# output probability
def output_prob(parameters, size, rep): 
    '''
    parameters from vqe, size should be (number of interations, size**2 * rep)
    size: size of the ising model 
    rep: number of rep in the ansatz
    '''
    ### assign parameter in the circuit 
    num_q = size**2
    qc = QuantumCircuit(num_q)
    for i in range(rep+1):
        for j in range(num_q): 
            qc.ry(parameters[j+i*num_q], j)
    
    ### measure 
    final_counts = backend.run(qc, shot=1000).result().get_statevector()
    probs = Statevector(final_counts).probabilities()
    # coeff is the square root of probs 
    coeff = np.sqrt(probs)
    
    # function for writing wave function in binary form
    def zero(a,num_q):
        string = ''
        if len(a) < num_q:
            for _ in range(num_q-len(a)):
                string += '0'
            string += a
            return string
        # add an else statement
        else:
            return a

    info = {}
    # the largest integer that format() can convert to binary is 255.
    for i in range(len(coeff)):
        info[zero(bin(i)[2:], num_q)] = np.round(coeff[i], 3)
    return info

In [5]:
G, w, pos = graph_generation(3)
max_cut = Maxcut(w)
qp = max_cut.to_quadratic_program()
qubitOp, offset = qp.to_ising()
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

In [6]:
print(qp)

maximize -2*x_0*x_1 - 2*x_0*x_3 - 2*x_1*x_2 - 2*x_1*x_4 - 2*x_2*x_5 - 2*x_3*x_4 - 2*x_3*x_6 - 2*x_4*x_5 - 2*x_4*x_7 - 2*x_5*x_8 - 2*x_6*x_7 - 2*x_7*x_8 + 2*x_0 + 3*x_1 + 2*x_2 + 3*x_3 + 4*x_4 + 3*x_5 + 2*x_6 + 3*x_7 + 2*x_8 (9 variables, 0 constraints, 'Max-cut')


In [7]:
def size9energy(solution):
    sol = [*solution]
    spin = np.array([eval(i) for i in sol])
    spin[spin == 0] = -1
    print(spin)
    x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8 = spin
    return -2*x_0*x_1 - 2*x_0*x_3 - 2*x_1*x_2 - 2*x_1*x_4 - 2*x_2*x_5 - 2*x_3*x_4 - 2*x_3*x_6 - 2*x_4*x_5 - 2*x_4*x_7 - 2*x_5*x_8 - 2*x_6*x_7 - 2*x_7*x_8+ 2*x_0 + 3*x_1 + 2*x_2 + 3*x_3 + 4*x_4 + 3*x_5 + 2*x_6 + 3*x_7 + 2*x_8 

In [8]:
size9energy('011100011')

[-1  1  1  1 -1 -1 -1  1  1]


14

In [9]:
def maxcut_obj(x, G):
    """
    Given a bitstring as a solution, this function returns
    the number of edges shared between the two partitions
    of the graph.
    
    Args:
        x: str
           solution bitstring
           
        G: networkx graph
        
    Returns:
        obj: float
             Objective
    """
    obj = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            obj -= 1
            
    return obj

In [10]:
maxcut_obj('101010101', G)

-12

In [12]:
file = '4x4 data seed2.xlsx'
n = 4
periodic = False
G, w, pos = graph_generation(n, periodic = periodic)
df = pd.read_excel(file)
sols = df['solution']
objective = []
sol_str = []
for sol in sols:
    sol = str(sol)
    if len(sol) != n**2:
        sol = '0'*(n**2 - len(sol)) + sol
    sol_str.append(sol)
    objective.append(maxcut_obj(sol, G))

df['max_cut objective'] = objective
df['solution'] = sol_str

writer = pd.ExcelWriter(file, engine = 'xlsxwriter')
df.to_excel(writer, sheet_name = 'different get_expectation seeds')
writer.save()
print('finish')

finish


In [51]:
'0' * 2

'00'